In [108]:
from sqlalchemy import create_engine
import geopandas as gp
import multiprocessing
import zipfile as zp
import pandas as pd
import numpy as np
import shutil
import os
from IPython.display import clear_output, display
import time
import concurrent.futures as cf

df = pd.read_csv(r'E:\combined.csv')
shp_df = df[df.Ext == 'shp']
shp_df.head()


# Function to read shape files into a temp directory
def shp_zip_read(full_path):
    os.makedirs(r"E:\New folder", exist_ok=True)
    zip_file_path = full_path.split('.zip\\')[0]
    shp_file_path = full_path.split('.zip\\')[1]
    archive = zp.ZipFile(zip_file_path)
    if '/' not in shp_file_path:
        for file in archive.namelist():
            if '/' not in file:
                archive.extract(file, r"E:\New folder")
    else:
        shpfile_folder_path = '/'.join(file_zip_path.split('/')[:-1])+'/'
        for file in archive.namelist():
            if file.startswith(shpfile_folder_path):
                archive.extract(file, r"E:\New folder")
    gdf = gp.read_file(r"E:\New folder\{}".format(file_zip_path))
    shutil.rmtree(r'E:\New folder', ignore_errors=True)
    return gdf


# Function to dump shape files into postgres db 
def shape_file_dump(file_name, file_path, engine):
    
    conn = engine.connect()
    
    # Reading the shape file
    if '.zip' in file_path:
        gdf = shp_zip_read(file_path)
    else:
        gdf = gp.read_file(file_path)

    #Import shapefile to databse
    #print(file_name)
    gdf.to_postgis(name=file_name, con=engine, schema="postgis")
    
    conn.close()
    

# Deriving a unique file name to create table in the database 
import hashlib
class FilePathShortener:
    def __init__(self):
        self.paths = {}
    
    def shorten_file_path(self, file_path):
        if file_path in self.paths:
            return self.paths[file_path]
        
        file_hash = hashlib.md5(file_path.encode()).hexdigest()
        shortened_path = file_hash[:8]
        
        self.paths[file_path] = shortened_path
        return shortened_path
    
    def decode_shortened_path(self, shortened_path):
        for path, short_path in self.paths.items():
            if short_path == shortened_path:
                return path
        
        return None

shortener = FilePathShortener()

 

"""file_path = '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'
shortened_path = shortener.shorten_file_path(file_path)
print(shortened_path)"""

shortened_path = shp_df.File_path.apply(shortener.shorten_file_path) 
new_file_name = shp_df.File_Name+'_'+shortened_path
shp_df['unique_file_name']=new_file_name
        
    
if __name__ == '__main__':
    # Number of processes to run in parallel
    num_processes = multiprocessing.cpu_count()
        
    #DB details
    host = 'localhost'
    port = '5432'
    database = 'postgis_33'
    user = 'postgres'
    password = 'wadhwanikdss'

    conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(conn)
            
#     # Create a list to store the process instances
#     processes = []

#     for _ in range(num_processes):
#         p = multiprocessing.Process(target=shape_file_dump, args=(list(shp_df.File_Name), list(shp_df.File_path), engine))
#         p.start()
#         processes.append(p)

#     # Wait for all processes to finish
#     for p in processes:
#         p.join()
    
    j = 1
    
    
    lst = []    
    for i in range(shp_df.shape[0]):
        print(f'Executing {i}st file')
        try:
            shape_file_dump(shp_df.unique_file_name.iloc[i], shp_df.File_path.iloc[i], engine)
            #print(shp_df.unique_file_name.iloc[i])
        except Exception as e:
            print(e, shp_df.File_path.iloc[i])
            lst.append(shp_df.File_path.iloc[i])

    print("Data dumped successfully into the PostGIS database.")
    
    engine.dispose()

C:\Users\user\AppData\Local\Temp\ipykernel_3452\1342919420.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df['unique_file_name']=new_file_name


Executing 8500st file
Table 'UK_10_POINTS_fbb000d9' already exists. \\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\final\New folder\UK_10_POINTS.shp
Executing 8501st file
Table 'UP_b44e1730' already exists. \\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\final\New folder\UP.shp
Executing 8502st file
Table 'UP2_63c9441c' already exists. \\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\final\New folder\UP2.shp
Executing 8503st file
Table 'UTTARKHAND_c4edc866' already exists. \\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\final\New folder\UTTARKHAND.shp
Executing 8504st file
Table 'AP_FINAL_2c3c7228' already exists. \\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\final\New folde

KeyboardInterrupt: 

In [93]:
shp_df.File_path.apply(lambda row : row if '.zip' in row else np.nan).dropna().shape

(1036,)

In [88]:
open('\\\\192.168.1.151\\ncfc_data3\\Akhilesh\\akhileshPP\\PP_work\\RD\\xacto_ArcGIS101_062014')

FileNotFoundError: [Errno 2] No such file or directory: '\\\\192.168.1.151\\ncfc_data3\\Akhilesh\\akhileshPP\\PP_work\\RD\\xacto_ArcGIS101_062014.zip\\input_files/freeburg_xs_line.shp'

In [3]:
shp_df.shape

(98875, 5)

In [34]:
shp_df.File_path.head()

19     \\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Co...
35     \\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Do...
58     \\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Do...
145    \\192.168.1.151\ncfc_data3\Akhilesh\akhileshPP...
152    \\192.168.1.151\ncfc_data3\Akhilesh\akhileshPP...
Name: File_path, dtype: object

In [17]:
from IPython.display import display

In [20]:
path='\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'

In [21]:
path

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'

In [19]:
for i in range(10):
    display(shp_df[shp_df.File_path.apply(lambda row:'0.India_state_shp' in row)].File_path.iloc[i])

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india_boundary_lcc.shp'

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india_dist_lcc.shp'

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\Others\\shp\\0.India_state_shp\\india state_lcc.shp'

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\Others\\shp\\0.India_state_shp\\india_boundary_lcc.shp'

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\Others\\shp\\0.India_state_shp\\india_dist_lcc.shp'

IndexError: single positional indexer is out-of-bounds

In [5]:
shp_df['File_path'].nunique()

98874

In [51]:
def clean(row):
    return '-'.join(row.split('\\')[-12:])
x = shp_df.File_path.apply(clean)


In [49]:
x.sort_values()

19         --192.168.1.151-ncfc_data3-Akhilesh-2018-19-Co...
35         --192.168.1.151-ncfc_data3-Akhilesh-2018-19-Do...
58         --192.168.1.151-ncfc_data3-Akhilesh-2018-19-Do...
717        --192.168.1.151-ncfc_data3-Akhilesh-Sugercane-...
988        --192.168.1.151-ncfc_data3-Akhilesh-Sugercane-...
                                 ...                        
1409684    ncfc_data3-PMFBY analysis-Kharif Rabi Pilot st...
1409685    ncfc_data3-PMFBY analysis-Kharif Rabi Pilot st...
1409686    ncfc_data3-PMFBY analysis-Kharif Rabi Pilot st...
1409687    ncfc_data3-PMFBY analysis-Kharif Rabi Pilot st...
1409688    ncfc_data3-PMFBY analysis-Kharif Rabi Pilot st...
Name: File_path, Length: 98875, dtype: object

In [36]:
shp_df['File_path'].nunique()

98874

In [52]:
x.nunique()

82429

In [42]:
display(x.head())

19     __192.168.1.151_ncfc_data3_Akhilesh_2018-19_Co...
35     __192.168.1.151_ncfc_data3_Akhilesh_2018-19_Do...
58     __192.168.1.151_ncfc_data3_Akhilesh_2018-19_Do...
145    __192.168.1.151_ncfc_data3_Akhilesh_akhileshPP...
152    __192.168.1.151_ncfc_data3_Akhilesh_akhileshPP...
Name: File_path, dtype: object

In [56]:
#Deriving short unique file path code 

import hashlib

class FilePathShortener:
    def __init__(self):
        self.paths = {}
    
    def shorten_file_path(self, file_path):
        if file_path in self.paths:
            return self.paths[file_path]
        
        file_hash = hashlib.md5(file_path.encode()).hexdigest()
        shortened_path = file_hash[:8]
        
        self.paths[file_path] = shortened_path
        return shortened_path
    
    def decode_shortened_path(self, shortened_path):
        for path, short_path in self.paths.items():
            if short_path == shortened_path:
                return path
        
        return None

shortener = FilePathShortener()

 

"""file_path = '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'
shortened_path = shortener.shorten_file_path(file_path)
print(shortened_path)"""

shortened_path = shp_df.File_path.apply(shortener.shorten_file_path)


decoded_path = shortener.decode_shortened_path(shortened_path)
print(decoded_path)

e621a0da
\\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Backup_Del\Transfer-data\Sent-Analysis\shp\0.India_state_shp\india state_lcc.shp


In [58]:
file_path = shp_df.File_path
shortened_path = shortener.shorten_file_path(file_path)
print(shortened_path)

TypeError: unhashable type: 'Series'

In [68]:
shortened_path = shp_df.File_path.apply(shortener.shorten_file_path)
new_file_name = shp_df.File_Name+'_'+shortened_path

In [62]:
shortened_path.nunique()


98873

In [65]:
new_file_name.nunique()

98874

In [79]:
shp_df['unique_file_name']=new_file_name

C:\Users\user\AppData\Local\Temp\ipykernel_3452\3839595209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df['unique_file_name']=new_file_name


In [80]:
shp_df.head()

,File_Name,Columns,Ext,Size(in mb),File_path,unique_file_name
19,Cotton_States,"{'STATE', 'geometry', 'State_1'}",shp,2.354198,\\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Co...,Cotton_States_41d30cad
35,Banswara,"{'ID', 'STATE_NAME', 'TALUK', 'geometry', 'SUM...",shp,0.347881,\\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Do...,Banswara_9e90aae4
58,Rajasthan,"{'ID', 'STATE_NAME', 'DIST_IMD', 'ID_RAIN', 'D...",shp,0.477604,\\192.168.1.151\ncfc_data3\Akhilesh\2018-19\Do...,Rajasthan_b0ad2740
145,Sirmaur,"{'VALIDTO_2', 'TYPE_2', 'Shape_Area', 'NAME_2'...",shp,0.007275,\\192.168.1.151\ncfc_data3\Akhilesh\akhileshPP...,Sirmaur_cce49301
152,sirmaur_lcc,"{'VALIDTO_2', 'TYPE_2', 'Shape_Area', 'NAME_2'...",shp,0.007275,\\192.168.1.151\ncfc_data3\Akhilesh\akhileshPP...,sirmaur_lcc_7f8a3da6


In [102]:
decoded_path = shortener.decode_shortened_path('639e1a68')
print(decoded_path)

\\192.168.1.151\ncfc_data3\Neetu Mam (IP 26 Data Backup 17 April 23)\Dr. Kanthesh\Desktop 5-5-2022\Shape Files\PILOT STUDY\village boundary\block_3_sst_vil\Blocks\9.shp


In [104]:
def func(x):
    import pandas as pd
    return pd.Series([x['C'].mean()])

df.groupby(["A","B"]).apply_parallel(func, num_processes=30)

KeyError: 'A'

In [112]:
lst

['\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UK_10_POINTS.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UP.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UP2.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UTTARKHAND.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\final\\AP_FINAL.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\final\\CCE_POINTS.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Back

In [113]:
lst

['\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UK_10_POINTS.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UP.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UP2.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\UTTARKHAND.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\final\\AP_FINAL.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Dr. Kanthesh\\Desktop 5-5-2022\\Shape Files\\final\\New folder\\final\\CCE_POINTS.shp',
 '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Back